<h1 style="text-align:center">Tugas MYSQL Kelompok 2</h1>

## Problem Statement

---
X-mart is our team’s latest venture and after running international operations for his online supermarket that specialises in fresh produce. The founder is asking for your support to analyse his sales performance. In June 2020, large-scale supply changes were made at Data Mart. All Data Mart products now use sustainable packaging methods in every single step from the farm all the way to the customer.

The founder needs your help to quantify the impact of this change on the sales performance for Data Mart and it’s separate business areas. The key business question he wants you to help him answer is the following:

- What was the quantifiable impact of the changes introduced in June 2020?
- Which platform, region, segment, and customer types were the most impacted by this change?
- What can we do about the future introduction of similar sustainability updates to the business to minimize the impact on sales?

## Available Data

---
For this case study there is only a single table `weekly_sales`. The columns are pretty self-explanatory based on the column names but here are some further details about the dataset:

1. Data Mart has international operations using a multi-`region` strategy
2. Data Mart has both, a retail and online `platform` in the form of a Shopify store front to serve their customers
3. Customer `segment` and `customer_type` data relates to personal age and demographics information that is shared with Data Mart
4. `transactions` is the count of unique purchases made through Data Mart and `sales` is the actual dollar amount of purchases

Each record in the dataset is related to a specific aggregated slice of the underlying sales data rolled up into a `week_date` value which represents the start of the sales week.

Link to download: [here](https://drive.google.com/drive/folders/1pprpd4NbbZ_gBfccCjAqbhF0aJKtoMI9)

## Case Study Questions

---
The following case study questions require some data cleaning steps before we start to unpack key business questions in more depth. In a single query, perform the following operations and generate a new table in the `data_mart` schema named `clean_weekly_sales`:

1. Convert the week_date to a DATE format.
2. Add a week_number as the second column for each `week_date` value, for example any value from the 1st of January to 7th of January will be 1, 8th to 14th will be 2 etc.
3. Add a `month_number` with the calendar month for each `week_date` value as the 3rd column.
4. Add a `calendar_year` column as the 4th column containing either 2018, 2019 or 2020 values.
5. Add a new column called `age_band` after the original `segment` column using the following mapping on the number inside the `segment` value:
    
 <center>   
    <table>
  <tr>
    <th>segment</th>
    <th>age_band</th>
  </tr>
  <tr>
    <td>1</td>
    <td>Young Adults</td>
  </tr>
  <tr>
    <td>2</td>
    <td>Middle Aged</td>
  </tr>
  <tr>
    <td>3 or 4</td>
    <td>Retirees</td>
  </tr>
</table>
</center>

6. Add a new `demographic` column using the following mapping for the first letter in the `segment` values:
    
<center>  
<table>
  <tr>
    <td>segment</td>
    <td>demographic</td>
  </tr>
  <tr>
    <td>C</td>
    <td>Couples</td>
  </tr>
  <tr>
    <td>F</td>
    <td>Families</td>
  </tr>
</table>
</center>
    
7. Ensure all `null` string values with an `"unknown"` string value in the original `segment` column as well as the new `age_band` and `demographic` columns
8. Generate a new `avg_transaction` column as the `sales` value divided by `transactions` rounded to 2 decimal places for each record

In [1]:
import mysql.connector
import pandas as pd

In [2]:
# define function for interacting with db
def show(db, query):
    con = mysql.connector.connect(
        host = 'localhost',
        user = 'root',
        passwd = '123456789',
        database = db
    )
    
    executor = con.cursor()
    executor.execute(query)
    table = executor.fetchall()
    df = pd.DataFrame(table, columns = executor.column_names)
    
    con.close()
    return df

# Case Study Questions

---
The following case study questions require some data cleaning steps before we start to unpack key business questions in more depth. In a single query, perform the following operations and generate a new table in the `data_mart` schema named `clean_weekly_sales`:

1. Convert the week_date to a DATE format.
2. Add a week_number as the second column for each `week_date` value, for example any value from the 1st of January to 7th of January will be 1, 8th to 14th will be 2 etc.
3. Add a `month_number` with the calendar month for each `week_date` value as the 3rd column.
4. Add a `calendar_year` column as the 4th column containing either 2018, 2019 or 2020 values.
5. Add a new column called `age_band` after the original `segment` column using the following mapping on the number inside the `segment` value:
    
 <center>   
    <table>
  <tr>
    <th>segment</th>
    <th>age_band</th>
  </tr>
  <tr>
    <td>1</td>
    <td>Young Adults</td>
  </tr>
  <tr>
    <td>2</td>
    <td>Middle Aged</td>
  </tr>
  <tr>
    <td>3 or 4</td>
    <td>Retirees</td>
  </tr>
</table>
</center>

6. Add a new `demographic` column using the following mapping for the first letter in the `segment` values:

<center>  
<table>
  <tr>
    <td>segment</td>
    <td>demographic</td>
  </tr>
  <tr>
    <td>C</td>
    <td>Couples</td>
  </tr>
  <tr>
    <td>F</td>
    <td>Families</td>
  </tr>
</table>
</center>

7. Ensure all `null` string values with an `"unknown"` string value in the original `segment` column as well as the new `age_band` and `demographic` columns
8. Generate a new `avg_transaction` column as the `sales` value divided by `transactions` rounded to 2 decimal places for each record

In [3]:
# drop table
q = ('''DROP TABLE IF EXISTS clean_weekly_sales''')
show('data_mart', query = q)

""


In [4]:
# create one-query table
q = ('''
CREATE TABLE data_mart.clean_weekly_sales AS 
    (SELECT STR_TO_DATE(week_date, '%d-%m-%y') AS week_date,
    WEEK(STR_TO_DATE(week_date, '%d-%m-%y')) AS week_number,
    MONTH(STR_TO_DATE(week_date, '%d-%m-%y')) AS month_number,
    YEAR(STR_TO_DATE(week_date, '%d-%m-%y')) AS calendar_year,
    region,
    platform,
    CASE
        WHEN segment = 'C3' THEN 'C3'
        WHEN segment = 'F1' THEN 'F1'
        WHEN segment = 'C1' THEN 'C1'
        WHEN segment = 'C2' THEN 'C2'
        WHEN segment = 'F2' THEN 'F2'
        WHEN segment = 'F3' THEN 'F3'
        WHEN segment = 'C4' THEN 'C4'
        WHEN segment = 'null' THEN 'Unknown'
    END AS segment,
    CASE
        WHEN RIGHT(segment, 1) = '1' THEN 'young adults'
        WHEN RIGHT(segment, 1) = '2' THEN 'Middle Aged'
        WHEN RIGHT(segment, 1) IN ('3' , '4') THEN 'Retirees'
        ELSE 'Unknown'
    END AS age_band,
    CASE
        WHEN LEFT(segment, 1) = 'C' THEN 'Couples'
        WHEN LEFT(segment, 1) = 'F' THEN 'Families'
        ELSE 'Unknown'
    END AS demographic,
    ROUND((sales / transactions), 2) AS avg_transactions,
    transactions,
    sales FROM
    weekly_sales);
''')
show('data_mart', query = q)

""


In [13]:
# check clean data
q = ('''SELECT * FROM clean_weekly_sales''')
show('data_mart', query = q).head(10)

,week_date,week_number,month_number,calendar_year,region,platform,segment,age_band,demographic,avg_transactions,transactions,sales
0,2020-08-31,35,8,2020,ASIA,Retail,C3,Retirees,Couples,30.31,120631,3656163.00
1,2020-08-31,35,8,2020,ASIA,Retail,F1,young adults,families,31.56,31574,996575.00
2,2020-08-31,35,8,2020,USA,Retail,Unknown,Unknown,Unknown,31.20,529151,16509610.00
3,2020-08-31,35,8,2020,EUROPE,Retail,C1,young adults,Couples,31.42,4517,141942.00
4,2020-08-31,35,8,2020,AFRICA,Retail,C2,Middle Aged,Couples,30.29,58046,1758388.00
5,2020-08-31,35,8,2020,CANADA,Shopify,F2,Middle Aged,families,182.54,1336,243878.00
6,2020-08-31,35,8,2020,AFRICA,Shopify,F3,Retirees,families,206.64,2514,519502.00
7,2020-08-31,35,8,2020,ASIA,Shopify,F1,young adults,families,172.11,2158,371417.00
8,2020-08-31,35,8,2020,AFRICA,Shopify,F2,Middle Aged,families,155.84,318,49557.00
9,2020-08-31,35,8,2020,AFRICA,Retail,C3,Retirees,Couples,35.02,111032,3888162.00


In [14]:
# check info
show('data_mart', query = q).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17117 entries, 0 to 17116
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   week_date         17117 non-null  object
 1   week_number       17117 non-null  int64 
 2   month_number      17117 non-null  int64 
 3   calendar_year     17117 non-null  int64 
 4   region            17117 non-null  object
 5   platform          17117 non-null  object
 6   segment           17117 non-null  object
 7   age_band          17117 non-null  object
 8   demographic       17117 non-null  object
 9   avg_transactions  17117 non-null  object
 10  transactions      17117 non-null  int64 
 11  sales             17117 non-null  object
dtypes: int64(4), object(8)
memory usage: 1.6+ MB


**Insights**:
- The DataFrame consists of 17117 rows and 12 columns.
- No missing values are found in the dataset as seen from the Non-Null Count.

## ***Question #1***

### 1. What day of the week is used for each week_date value?

In [15]:
q = ('''SELECT week_date, DAYNAME(week_date) AS day_of_week
FROM clean_weekly_sales''')
day_of_week = show('data_mart', query = q)
day_of_week.sample(10)

,week_date,day_of_week
12015,2018-08-20,Monday
10282,2019-04-22,Monday
5443,2020-03-30,Monday
8936,2019-06-03,Monday
3075,2020-06-08,Monday
5880,2019-09-02,Monday
8647,2019-06-10,Monday
940,2020-08-10,Monday
12545,2018-08-06,Monday
6072,2019-08-26,Monday


In [16]:
day_of_week['day_of_week'].unique()

array(['Monday'], dtype=object)

**Insight**:
- The column `day_of_week` column only consists of Monday only because every date in the column `week_date` is Monday.

### 2. What range of week numbers are missing from the dataset?

In [17]:
q = '''
SELECT DISTINCT week_number
FROM clean_weekly_sales;
'''
show('data_mart', query=q).head(10)

,week_number
0,35
1,34
2,33
3,32
4,31
5,30
6,29
7,28
8,27
9,26


**Insight**:
- There is no missing week number in the dataset.

### 3. How many total transactions were there for each year in the dataset?

In [18]:
q = '''
SELECT calendar_year,
       SUM(transactions) AS total_transactions
FROM clean_weekly_sales
GROUP BY calendar_year
ORDER BY calendar_year;
'''
total_transaction = show('data_mart', query=q)
total_transaction

,calendar_year,total_transactions
0,2018,346406460
1,2019,365639285
2,2020,375813651


**Insight**: The annual `total_transaction` is relatively similar between 2018 and 2020 but then the value dropped significantly in 2020.

### 4. What is the total sales for each region for each month?

In [36]:
q = '''
SELECT 
    region,
    calendar_year as year,
    month_number,
    SUM(sales) AS total_sales
FROM 
    clean_weekly_sales
GROUP BY 
    region,
    calendar_year,
    month_number
ORDER BY 
	calendar_year,
	month_number,
    region;
'''
total_sales = show('data_mart', query=q)
total_sales.head()

,region,year,month_number,total_sales
0,AFRICA,2018,3,130542213.00
1,ASIA,2018,3,119180883.00
2,CANADA,2018,3,33815571.00
3,EUROPE,2018,3,8402183.00
4,OCEANIA,2018,3,175777460.00


### 5. What is the total count of transactions for each platform?

In [37]:
q = '''
SELECT 
    platform,
    SUM(transactions) AS total_transactions
FROM 
    clean_weekly_sales
GROUP BY 
    platform;
'''
total_transaction = show('data_mart', query=q)
total_transaction

,platform,total_transactions
0,Retail,1081934227
1,Shopify,5925169


**Insight**: The Retail platform has significantly more transactions compared to the Shopify with over a billion transactions recorded for Retail and approximately 5.9 million transaction for Shopify.

### 6. Which `age_band` and `demographic` values contribute the most to Retail sales?

In [ ]:
q = '''
SELECT 
    age_band,
    demographic,
    SUM(sales) AS total_sales
FROM 
    clean_weekly_sales
WHERE 
    platform = 'Retail'
GROUP BY 
    age_band, demographic
ORDER BY 
    total_sales DESC
LIMIT 5;
'''
retail = show('data_mart', query=q)
retail

,age_band,demographic,total_sales
0,Unknown,Unknown,16067285533.00
1,Retirees,families,6634686916.00
2,Retirees,Couples,6370580014.00
3,Middle Aged,families,4354091554.00
4,young adults,Couples,2602922797.00


**Insight**: 
- The total sales is predominated by Unknown `demographic` and `age_band`.

### 7. Can we use the `avg_transaction` column to find the average transaction size for each year for Retail vs Shopify? If not - how would you calculate it instead?

In [ ]:
q = '''
SELECT 
    calendar_year AS year,
    platform,
    AVG(avg_transactions) AS avg_of_avg_transaction,
    SUM(sales)/SUM(transactions) AS avg_transaction_size
FROM 
    clean_weekly_sales
WHERE 
    platform IN ('Retail', 'Shopify')
GROUP BY 
    calendar_year, platform;
'''
avg_transaction_size = show('data_mart', query=q)
avg_transaction_size

,year,platform,avg_of_avg_transaction,avg_transaction_size
0,2020,Retail,40.640231,36.556573
1,2020,Shopify,174.873569,179.033210
2,2019,Retail,41.968071,36.833455
3,2019,Shopify,177.559562,183.361069
4,2018,Retail,42.906369,36.562650
5,2018,Shopify,188.279272,192.481312


**Insight**:
- Unable to find transaction size for each year using `avg_transaction`.
- Transaction size is better be computed by using `SUM(sales)/SUM(transactions)`

## ***Question #2: Before & After Analysis***

This technique is usually used when we inspect an important event and want to inspect the impact before and after a certain point in time. 

Taking the `week_date` value of `2020-06-15` as the baseline week where the Data Mart sustainable packaging changes came into effect. We would include all `week_date` values for `2020-06-15` as the start of the period **after** the change and the previous `week_date` values would be **before.** 

Using this analysis approach - answer the following questions:

### 1. What is the total sales for the 4 weeks before and after `2020-06-15`? What is the growth or reduction rate in actual values and percentage of sales?

In [ ]:
q = '''
WITH sales_before AS (
    SELECT 
        SUM(sales) AS total_sales
    FROM 
        clean_weekly_sales
    WHERE 
        week_date >= DATE_SUB('2020-06-15', INTERVAL 4 WEEK) AND week_date < '2020-06-15'
), sales_after AS (
    SELECT 
        SUM(sales) AS total_sales
    FROM 
        clean_weekly_sales
    WHERE 
        week_date >= '2020-06-15' AND week_date <= DATE_ADD('2020-06-15', INTERVAL 4 WEEK)
)
SELECT 
    'Before' AS period,
    sales_before.total_sales AS total_sales_before,
    'After' AS period,
    sales_after.total_sales AS total_sales_after,
    (sales_after.total_sales - sales_before.total_sales) AS absolute_change,
    ((sales_after.total_sales - sales_before.total_sales) / sales_before.total_sales) * 100 AS percentage_change
FROM 
    sales_before, sales_after;
'''
week_4 = show('data_mart', query=q)
week_4

,period,total_sales_before,period,total_sales_after,absolute_change,percentage_change
0,Before,2345878357.00,After,2904930571.00,559052214.00,23.831253


**Insights**:
- After 2020-06-15, there was a significant increase in total sales compared to the period before.
- The total sales increased by approx 559M units, which represent 24% growth in sales compared to the period before 2020-06-15.

### 2. What about the entire 12 weeks before and after?

In [ ]:
q = '''
WITH sales_before AS (
    SELECT 
        SUM(sales) AS total_sales
    FROM 
        clean_weekly_sales
    WHERE 
        week_date >= DATE_SUB('2020-06-15', INTERVAL 12 WEEK) AND week_date < '2020-06-15'
), sales_after AS (
    SELECT 
        SUM(sales) AS total_sales
    FROM 
        clean_weekly_sales
    WHERE 
        week_date >= '2020-06-15' AND week_date <= DATE_ADD('2020-06-15', INTERVAL 12 WEEK)
)
SELECT 
    'Before' AS period,
    sales_before.total_sales AS total_sales_before,
    'After' AS period,
    sales_after.total_sales AS total_sales_after,
    (sales_after.total_sales - sales_before.total_sales) AS absolute_change,
    ((sales_after.total_sales - sales_before.total_sales) / sales_before.total_sales) * 100 AS percentage_change
FROM 
    sales_before, sales_after;
'''
week_12 = show('data_mart', query=q)
week_12

,period,total_sales_before,period,total_sales_after,absolute_change,percentage_change
0,Before,7126273147.00,After,6973947753.00,-152325394.00,-2.137518


**Insights**: The total sales decreased in the 12 weeks before the sustainable packaging changes to 69B in the 12 weeks after the changes. Moreover, the total sales decreased by approx 2% after the sustainable packaging changes were implemented.

### 3. How do the sale metrics for these 2 periods before and after compare with the previous years in 2018 and 2019?

INTERVAL 4 WEEKS

In [ ]:
q = '''
WITH cte AS (
  SELECT DISTINCT week_number FROM clean_weekly_sales
  WHERE week_date = '2020-06-15'
 ),
 four_weeks_before AS (
  SELECT DISTINCT week_date
  FROM clean_weekly_sales
  WHERE week_number BETWEEN (SELECT week_number FROM cte) - 4 AND (SELECT week_number FROM cte) - 1
 ),
 four_weeks_after AS (
  SELECT DISTINCT week_date
  FROM clean_weekly_sales
  WHERE week_number BETWEEN (SELECT week_number FROM cte) AND (SELECT week_number FROM cte) + 3
 ),
 summations AS (
  SELECT calendar_year, SUM(CASE WHEN week_date IN (SELECT * FROM four_weeks_before) THEN sales END) AS four_weeks_before,
  SUM(CASE WHEN week_date IN (SELECT * FROM four_weeks_after) THEN sales END) AS four_weeks_after
  FROM clean_weekly_sales
  GROUP BY calendar_year
 )
 SELECT *,
  four_weeks_after - four_weeks_before AS variance,
  ROUND(100 * (four_weeks_after - four_weeks_before)/four_weeks_before, 2) AS percentage_change
 FROM summations
 ORDER BY calendar_year;
'''
interval_4_weeks = show('data_mart', query=q)
interval_4_weeks

,calendar_year,four_weeks_before,four_weeks_after,variance,percentage_change
0,2018,2125140809.00,2129242914.00,4102105.00,0.19
1,2019,2249989796.00,2252326390.00,2336594.00,0.10
2,2020,2345878357.00,2318994169.00,-26884188.00,-1.15


**Insights**: 
- The table shows a consistent pattern across the years, with marginal fluctuations in sales metrics before and after the sustainable packaging changes.
- But in 2020, a notable decrease in total sales of approx 1% occurred following the implementation of changes contrasting with the minor variances observed in the preceding years.

INTERVAL 12 WEEKS

In [ ]:
q = '''
WITH cte AS (
  SELECT DISTINCT week_number FROM clean_weekly_sales
  WHERE week_date = '2020-06-15'
),
twelve_weeks_before AS (
  SELECT DISTINCT week_date
  FROM clean_weekly_sales
  WHERE week_number BETWEEN (SELECT week_number FROM cte) - 12 AND (SELECT week_number FROM cte) - 1
),
twelve_weeks_after AS (
  SELECT DISTINCT week_date
  FROM clean_weekly_sales
  WHERE week_number BETWEEN (SELECT week_number FROM cte) 
    AND (SELECT week_number FROM cte) + 11
),
summations AS (
SELECT calendar_year, 
  SUM(CASE 
        WHEN week_date in (select * from twelve_weeks_before) 
        THEN sales 
      END) AS twelve_weeks_before,
  SUM(CASE 
        WHEN week_date in (select * from twelve_weeks_after) 
        THEN sales 
      END) AS twelve_weeks_after
FROM clean_weekly_sales
GROUP BY calendar_year
)
SELECT *,
  twelve_weeks_after - twelve_weeks_before AS variance,
  ROUND(100 * (twelve_weeks_after - twelve_weeks_before)
  /twelve_weeks_before, 2) AS percentage_change
FROM summations
ORDER BY calendar_year;
'''
interval_12_weeks = show('data_mart', query=q)
interval_12_weeks

,calendar_year,twelve_weeks_before,twelve_weeks_after,variance,percentage_change
0,2018,6396562317.00,6500818510.00,104256193.00,1.63
1,2019,6883386397.00,6862646103.00,-20740294.00,-0.30
2,2020,7126273147.00,6973947753.00,-152325394.00,-2.14


**Insights**:
- The analysis of 12-weeks sales intervals shows consistent growth in 2018 and 2019 with a slight increase and decrease with a slight increase and decrease respectively.
- However, there was a significant drop of approx 2% in 2020 following sustainable packaging changes.

<h1 style="text-align:center"><i>---End---</i></h1>